In [1]:
import pandas as pd
import json

In [2]:
train_df = pd.read_csv('train.csv',header=0)
train_df = train_df[:10002]

In [3]:
train_df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
df_viz = train_df[['X', 'Y']]

In [5]:
df_viz.head()

,X,Y
0,-122.425892,37.774599
1,-122.425892,37.774599
2,-122.424363,37.800414
3,-122.426995,37.800873
4,-122.438738,37.771541


In [18]:
#Converts a dataframe to a geojson Point output
def df_to_geojson_point(df, properties, lat='X', lon='Y'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [19]:
#Converts a dataframe to a geojson LineString output
def df_to_geojson_line(df, properties, coords):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'LineString',
                               'coordinates': ''}}
        feature['geometry']['coordinates'] = row[coords]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [20]:
from IPython.display import Javascript
from IPython.display import HTML

In [9]:
geojson =  { "type": "FeatureCollection",
    "features": [
      { "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [102.0, 0.5]},
        "properties": {"prop0": "value0"}
        },
      { "type": "Feature",
        "geometry": {
          "type": "LineString",
          "coordinates": [
            [102.0, 0.0], [103.0, 1.0], [104.0, 0.0], [105.0, 1.0]
            ]
          },
        "properties": {
          "prop0": "value0",
          "prop1": 0.0
          }
        },
      { "type": "Feature",
         "geometry": {
           "type": "Polygon",
           "coordinates": [
             [ [100.0, 0.0], [101.0, 0.0], [101.0, 1.0],
               [100.0, 1.0], [100.0, 0.0] ]
             ]
         },
         "properties": {
           "prop0": "value0",
           "prop1": {"this": "that"}
           }
         }
       ]
     }

In [24]:
geojson = df_to_geojson_point(df_viz, properties=[])

In [10]:
#Create a javascript variable with our geojson data to visualize in the browser
#The data object 'vizObj' will be a global varialbe in our window that 
#We can pass to another javascript function call
Javascript("""window.vizObj={};""".format(geojson))

<IPython.core.display.Javascript object>

In [11]:
#Create some HTML to style out map and define it's output size

HTML("""
<style> #map {
  position: relative;
  width: auto;
  height: 650px;
  overflow:visible;
}
</style>
""")

In [12]:
%%javascript
require.config({
  paths: {
      mapboxgl: 'https://api.tiles.mapbox.com/mapbox-gl-js/v0.16.0/mapbox-gl',
      bootstrap: 'https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min'
  }
});

<IPython.core.display.Javascript object>

In [13]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;
require(['mapboxgl', 'bootstrap'], function(mapboxgl, bootstrap){
    mapboxgl.accessToken = 'pk.eyJ1IjoiZmFuaWZpc2giLCJhIjoiY2luMmU4eW1uMGJlcXdibHUydzd2Z3d4MSJ9.65eMcvcCquqHHgDC9Qi3pA';
    var map = new mapboxgl.Map({
        container: 'map', // container id
        style: 'mapbox://styles/mapbox/dark-v8', //stylesheet location
        center: [-89.948470, 40.783860], // starting position
        zoom: 10 // starting zoom 
    });
    
    
    function addSegLayer(mapid) {
        // Mapbox GL JS Api - import segment
        var segment_src = new mapboxgl.GeoJSONSource({
            data: window.vizObj,
            maxzoom: 18,
            buffer: 1,
            tolerance: 1
        });
        try {
            mapid.addSource('segment', segment_src);
            mapid.addLayer({
                id: 'segment',
                type: 'line',
                source: 'segment',
                paint: {
                    "line-opacity": 1,
                    "line-width": 5,
                    "line-color": 'red',
                }
            });
        } catch (err) {
            console.log(err);
        }
    };
    
    map.once('style.load', function(e) {
        addSegLayer(map);
        map.addControl(new mapboxgl.Navigation({
            position: 'top-left'
        }));
    });
    
});
element.append("<div id='map'></div>");

<IPython.core.display.Javascript object>

In [15]:
import numpy as np

In [6]:
df_viz = train_df[(train_df.Category == 'ASSAULT')]
#df[(df.A == 1) & (df.D == 6)]

In [7]:
df_viz.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
17,2015-05-13 21:55:00,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Wednesday,INGLESIDE,NONE,GODEUS ST / MISSION ST,-122.421682,37.742822
51,2015-05-13 19:33:00,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Wednesday,BAYVIEW,NONE,23RD ST / WISCONSIN ST,-122.398696,37.754746
109,2015-05-13 17:47:00,ASSAULT,CHILD ABUSE (PHYSICAL),Wednesday,BAYVIEW,NONE,0 Block of WHITFIELD CT,-122.381838,37.731104
114,2015-05-13 17:40:00,ASSAULT,THREATS AGAINST LIFE,Wednesday,CENTRAL,NONE,1400 Block of STOCKTON ST,-122.409032,37.799253
165,2015-05-13 15:40:00,ASSAULT,"BATTERY, FORMER SPOUSE OR DATING RELATIONSHIP",Wednesday,PARK,NONE,1700 Block of MCALLISTER ST,-122.440880,37.777532


In [8]:
df_viz = df_viz[['X', 'Y']]

In [9]:
df_viz.head()

,X,Y
17,-122.421682,37.742822
51,-122.398696,37.754746
109,-122.381838,37.731104
114,-122.409032,37.799253
165,-122.440880,37.777532


In [10]:
labels = 'ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS'.split(',')

In [11]:
viz_data = {}
for l in labels:
    df_viz = train_df[(train_df.Category == l)]
    viz_data[l] = [ df_viz['Y'][:10000],  df_viz['X'][:10000]] 

In [13]:
from bokeh.io import output_file, show
from bokeh.models.widgets import Select
from bokeh.io import output_file, show, vform
from bokeh.plotting import show, output_notebook,figure
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, CustomJS, ResizeTool
)
map_options = GMapOptions(lat=37.7756, lng=-122.4193, map_type="roadmap", zoom=13)

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options, title="Austin"
)

data_source = ColumnDataSource(
    data=dict(
        all_data=viz_data
    )
)

source = ColumnDataSource({'lat':viz_data['ASSAULT'][0],'lon':viz_data['ASSAULT'][1]})


callback = CustomJS(args=dict(source=source, data_source=data_source), code="""
        var data = source.get('data');
        var all_data = data_source.get('all_data')
        var f = cb_obj.get('value')
        x = data['lat']
        y = data['lon']
        new_plot = all_data[f]
        for (i = 0; i < new_plot.length; i++) {
            x[i] = new_plot[0][i] 
            y[i] = new_plot[1][i]
        }
        source.trigger('change');
    """)

select = Select(title="Option:", value="ASSAULT", options=['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY', 'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE', 'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT' , 'EXTORTION', 'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING', 'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING', 'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES', 'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE', 'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE', 'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE,SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT', 'WARRANTS', 'WEAPON LAWS'], callback=callback)

circle = Circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)
#plot.add_layout(select)
pan = PanTool()
wheel_zoom = WheelZoomTool()                                                                                                                                                                    
plot.add_tools(pan, wheel_zoom, BoxSelectTool(), ResizeTool())

output_file("gmap_plot.html")
layout = vform(select, plot)
output_notebook()
show(layout)

Loading BokehJS ...

In [29]:
import pandas as pd
from bokeh.models.glyphs import Circle
from bokeh.plotting import show, output_notebook,figure
from bokeh.models import (
    GMapPlot, GMapOptions, Range1d, ColumnDataSource, LinearAxis,
    PanTool, WheelZoomTool,HoverTool, TapTool, OpenURL)
output_notebook()

Loading BokehJS ...